# Тестовое задание Стажёр в команду CoreML
### Маслов Михаил
#### Linux, 16G RAM (+6G swap) 

In [14]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from pprint import pprint

In [15]:
import os

DATA_DIR = r'/home/mika/JupyterNotebooks/Vk_test_ml/data/'
os.chdir(DATA_DIR)

# использую для всех случайных процессов чтобы можно было воспроизвести результаты
RNG_SEED = 42

# для встроенного распараллеливания библиотеки, не уверен что работает
os.environ['MKL_THREADING_LAYER'] = 'tbb'
os.environ['LK_NUM_PROCS'] = '8,4'
os.environ['NUMBA_NUM_THREADS'] = '6'

In [16]:
rating = pd.read_csv('rating.csv')

In [17]:
print('rating shape:\t', rating.shape)
print('min rating:\t',rating['rating'].min())
print('max rating:\t',rating['rating'].max())
print('uniq users count:', rating['userId'].unique().size)
print('uniq movies count:', rating['movieId'].unique().size)
rating.head()

rating shape:	 (20000263, 4)
min rating:	 0.5
max rating:	 5.0
uniq users count: 138493
uniq movies count: 26744


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


Также стоит отметить, что распределение количества информации известной о пользователях - неравномерное, <br>оно похоже на распределние <a href="https://en.wikipedia.org/wiki/Power_law">степенного закона</a>.<br>
Чтобы не загромождать ноутбук графиками, я хотел бы сослаться на неплохую <a href='https://www.kaggle.com/code/saadmuhammad17/data-analysis-of-movielens-25m-dataset'>визуализацию</a> датасета с kaggle.

## Eval metrics: RMSE & NDCG

<p style="font-size: 20">
Обычно для измерения точности рекомнедации используют <i>RMSE</i>, как например в <i>Netflix Prize</i>.<br> 
Однако достижение хороших показателей с точки зрения <i>RMSE</i> не всегда гарантирует хорошие показатели рекомендательной системы.<br> 
Также можно добавить, что рейтинг это порядковые данные, то есть 5-3 != 3-1, 
а "<i>чистый</i>" <i>RMSE</i> не учитывет не линейность рейтингов<br><br>
Поэтому для измерения качества рекомендательной системы мы также будем использовать метрику <i>NDCG</i>, <br>которая позволит нам оценить соответсвие "<i>идеальной</i>" рекомендации.
</p>

## Split train validate

переписать мб

Так как наша задача это предсказываение рейтинга фильмов для пользователей, то мы разобьём датасет для каждого пользователя. <br> Также при разбиение мы учтём время, хоть и не все модели его использует, но логично предположить, <br>что время это полезный признак имеющий смысл. <br>
Я взял 5 разбиений для кросс валидации, потому что это наиболее популярный варинат и увеличение или умененьшение этого числа не должно дать существенных изменений.<br>
Также я взял 5 последних фильмов у каждого пользователя для валидации, так как минимальное колчиество оценок у пользователя 20 и, как мне кажется, при оценки ранжироврованого списка фильмов оценка бует объективнее, если мы будем угадывать для всех одинаковое число фильмов.

In [ ]:
import lenskit.crossfold as xf

N_SPLITS = 5

rating = rating.rename(columns={'userId': 'user', 'movieId': 'item'})
for i, tp in enumerate(xf.partition_users(rating, N_SPLITS, xf.LastN(5), rng_spec=RNG_SEED)):
    tp.train.to_csv('20m.train-%d.csv' % (i,))
    tp.test.to_csv('20m.test-%d.csv' % (i,))

## Collaborative filtering

### Метод LFM (SVD-like <a href="https://sifter.org/~simon/journal/20061211.html">FunkSVD</a>)

Алгоритм:
<p>
Данный метод использует векторное представление пользователя и объекта,<br>
а также средний рейтинг пользователя и объекта<br>
С помощью градиентного спуска мы находим векторы для каждого пользователя и объекта.<br>
Важной частью этого алгоритма является L2-регуляризация, она предотварщает модель от переобучения, что является проблемой SVD++. Можно отметить, что L1-регуляризация не даёт качественого прироста в точности
</p>
<p>
    
Гиперпараметрами алгоритма являются:
- количество эпох и/или эпсилон изменения ошибки
- количество признаков для предстваления пользователя и объекта

</p>

<p>
<img src='imgs/svd.png'alt="без регуляризации">
<i style="float:right;">без регуляризации</i>

<img src='imgs/funksvd.png'>
<i style="float:right;">с регуляризацией</i>
<br>
</p>

Далее прочитав статью <a href="https://sifter.org/~simon/journal/20061211.html">Simon Funk</a> и проанализировав <a href="https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data">датасет</a> с <i>Netflix Prize</i>, я решил что наши данные очень схожи и <br> поэтому можно взять гиперпараметры из блога призёра этого исторического конкурса.<br><br>
А именно:
- количество эпох 120
- количество признаков 40

In [18]:
%%time
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.metrics.predict import rmse, global_metric
from lenskit.topn import ndcg
from joblib import Parallel, delayed
import psutil

results = []

# Можно снизить N_SPLITS c 5 
N_SPLITS = 1
n_features = 40
iterations = 120

def train_test_eval_svd(i):
    cf_train = pd.read_csv('20m.train-%d.csv' % (i,))
    cf_test =  pd.read_csv('20m.test-%d.csv' % (i,))
    
    cf_model_svd = FunkSVD(features=n_features, iterations=iterations, range=(0.5,5))
    cf_model_svd.fit(cf_train)
    
    # Предсказываем
    cf_pred = cf_model_svd.predict(cf_test)
    cf_test['prediction'] = cf_pred

    # Оцениваем
    cf_model_ndcg = ndcg(cf_test.rename(columns={'rating': 'original_rating','prediction': 'rating'}), cf_test)
    cf_model_rmse = global_metric(cf_test, metric=rmse)

    result = {
        'n_features': n_features,
        'n_epochs': iterations,
        'rmse': cf_model_rmse,
        'ndcg': cf_model_ndcg,
    }
    
    return result

# работа с процессами для библиотеки
current_process = psutil.Process()
subproc_before = set([p.pid for p in current_process.children(recursive=True)])

# на больше n_jobs ОЗУ не хватает
results = Parallel(n_jobs=3, backend='multiprocessing')(
    delayed(train_test_eval_svd)(i) for i in tqdm(range(N_SPLITS)))

# особенность библиотеки чтобы завершить выполнение
subproc_after = set([p.pid for p in psutil.Process().children(recursive=True)])
for subproc in subproc_after - subproc_before:
    psutil.Process(subproc).terminate()


  0%|          | 0/1 [00:00<?, ?it/s]

BLAS using multiple threads - can cause oversubscription
found 1 potential runtime problems - see https://boi.st/lkpy-perf


CPU times: user 151 ms, sys: 114 ms, total: 265 ms
Wall time: 19min 45s


In [19]:
avg_rmse = sum([res['rmse'] for res in results])/len(results)
avg_ndcg = sum([res['ndcg'] for res in results])/len(results)
print('Avg RMSE:', avg_rmse)
print('Avg NDCG:', avg_ndcg)

Avg RMSE: 0.8577087331709268
Avg NDCG: 0.9694998266280797


## Collaborative + content filtering

На этот раз я отказался от использования самописных нейроных сетей и взял библиотеку LightFM, которая позволяет использовать контентые признаки. В качестве контентного признака я выбрал жанры фильмов, по сути это категориальный признак, но в текстовом формате, и у одного объекта может быть несколько категорий.

По описанию библиотеки LightFM использует матричную факторизацию для решения задачи рекомендации, но с возможностью использовать не только user-item призанки.

### Подбор гиперпарметоров
Я уменьшил количество латентных факторов в два раза по сравнению с прошлой моделью в угоду производительности с 40 жо 20, так как надеюсь, что контентый признак компенсирует потери, а также количество обучающих эпох я взял 20 вместо 120 по тем же причинам.

In [1]:
%reset -f

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from pprint import pprint
import os
from datetime import datetime
import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

In [2]:
DATA_DIR = r'/home/mika/JupyterNotebooks/Vk_test_ml/data/'
os.chdir(DATA_DIR)

# использую для всех случайных процессов чтобы можно было воспроизвести результаты
RNG_SEED = 42

movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv').merge(movie)
n_features = 20

In [3]:
display('rating', rating.head())
display('movie', movie.sample(5))

'rating'

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,1996-12-25 15:26:09,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,1996-11-27 08:19:02,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,1996-06-23 20:36:14,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,1996-10-28 13:29:44,Jumanji (1995),Adventure|Children|Fantasy


'movie'

,movieId,title,genres
12219,55844,Itty Bitty Titty Committee (2007),Comedy|Drama|Romance
19057,94813,Chernobyl Diaries (2012),Horror
15619,79525,Human Desire (1954),Drama|Film-Noir
16884,85374,Emma (1932),Comedy|Drama|Romance
2234,2319,Reach the Rock (1998),Comedy|Drama


In [4]:
import itertools
movie_genre = [x.split('|') for x in rating['genres']]
all_movie_genre = sorted(list(set(itertools.chain.from_iterable(movie_genre))))

In [5]:
all_movie_genre

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [6]:
dataset = Dataset()
dataset.fit(rating['userId'], 
            rating['movieId'], 
            item_features=all_movie_genre)

In [7]:
item_features = dataset.build_item_features(
    (x, y) for x,y in zip(rating['movieId'], movie_genre))

In [9]:
(interactions, weights) = dataset.build_interactions(rating[['userId', 'movieId', 'rating']].values)
uid_map, ufeature_map, iid_map, ifeature_map = dataset.mapping()

У меня не хватает памяти для одноврменного хранения всех данных, поэтому очистим те что уже не нужны

In [17]:
del rating, dataset
%reset_selective -f rating dataset

In [13]:
train = pd.read_csv('20m.train-0.csv').rename(columns={'user':'userId', 'item': 'movieId'}).merge(movie)
train_dataset = Dataset()
train_dataset.fit(train['userId'], 
            train['movieId'], 
            item_features=all_movie_genre)

(train_interactions, t_weights) = train_dataset.build_interactions(train[['userId', 'movieId', 'rating']].values)

In [14]:
hybrid_model = LightFM(loss='warp', no_components=n_features, 
                 learning_rate=0.1, 
                 item_alpha=1e-6,
                 user_alpha=1e-6,
                 random_state=np.random.RandomState(RNG_SEED))

In [15]:
%%time
hybrid_model.fit(interactions=train_interactions,
           item_features=item_features,
           epochs=20, num_threads=32)

CPU times: user 1h 38min 5s, sys: 1min 34s, total: 1h 39min 39s
Wall time: 8min 42s


In [18]:
test = pd.read_csv('20m.test-0.csv').rename(columns={'user':'userId', 'item': 'movieId'}).merge(movie)

users, items, preds = [], [], []
item = list(test.movieId.unique())
for user in test.userId.unique():
    user = [user] * len(item)
    users.extend(user)
    items.extend(item)
test["uid"] = test.userId.map(uid_map)
test["iid"] = test.movieId.map(iid_map)
test["prediction"] = test.apply(
        lambda x: hybrid_model.predict(
            user_ids=np.array([x["uid"]], dtype=np.int32),
            item_ids=np.array([x["iid"]], dtype=np.int32),
            item_features=item_features,
            num_threads=32,
        )[0],
        axis=1,
    )

In [20]:
from lenskit.topn import ndcg
from lenskit.metrics.predict import rmse, global_metric

hybrid_ndcg = ndcg(test.rename(columns={'rating': 'original_rating','prediction': 'rating', 'movieId':'item', 'userId':'user'}), test)
hybrid_rmse = global_metric(test, metric=rmse)

print('RMSE:', hybrid_rmse)
print('NDCG:', hybrid_ndcg)

RMSE: 1.3780868573072507
NDCG: 0.9780378327939213


## Вывод

CF модель показала неплохие результаты, имея малое число признаков, но намного больше времени для обучения(~6 раз). Гибридная модель имела меньшее время обучения и параметров описывающих признаки, но показала результат даже лучше по метрике NDCG, но проиграла значительно в точном предсказание рейтнига.<br><br>
Я считаю, что NDCG более бизнесовая метрика, то есть показывает результат который больше связан с задачами бизнеса, а именно что лучше порекомендовать. В таком случае нам выгоднее использовать гибридный рекомендатор, но я думаю, что такой результат был очевиден заранее, так как понято, что контентные признаки несут в себе полезную информацию и дадут нашей модели больше точек опоры и следовательно большую точность. Также не стоит забывать что конекретная CF модель как раз старалась минимизировать RMSE, за счёт чего и имеет лучший показатель по ней, но RMSE не свосем честная метрика, так как рейтинг это порядковая величина(то есть 5-4 != 2-1), а использая RMSE мы как будто забываем про это.<br>



<!-- Хоть моя попытка сделать контентную модель провалилась, в техническом плане, всё равно можно сказать, что контентные признаки несут в себе полезную информацию и точность у такой модели должна быть выше, чем у чисто коллоборативной.<br><br>
Также в будущем можно улучшить точность обеих моделей в выбранных метриках, если заставить модели оптимизировать именно эти метрики, но целесообразность этого стоит проверить на A/B тестах. Не всегда хорошие значения в метрике, значит что рекомендательная система хорошо работает, онлайн метрики в совокупности с офлайн дают более полной представлнение о качестве работы системы.
<br><br> -->

<!-- P.S.:<br>
Я потратил слишком много времени и сил на кросс валидацию и копание в моделях, чем на выполнение самого задание.<br> Сейчас очевидо, что подбирать и брать большие гиперпараметры это не такая значимая часть исследования.<br> Лучшее несколько теоритических моделей, чем одна кроссвалидированая)<br>
Попробую переделать контентную модель, но уже в не рамках зачёта.<br>
 -->